In [10]:
import os
import requests

In [4]:
def akv_query_FS_graphql(query, variable):
    try:
        response = requests.post(
            "https://api-test.fellesstudentsystem.no/graphql/", 
            json={"query": query, "variables": variable},
            headers={'Authorization': f'Basic {os.environ["tokenFS"]}', "Feature-Flags": "beta, experimental"},
            )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as exc:
        raise exc


In [11]:
spørjing = """
query MyQuery($plnr: [String!]) {
  fagpersonerGittPersonlopenumre(
    personlopenumre: $plnr
    eierOrganisasjonskode: "0203"
  ) {
    aktiv
    id
    arbeidsadresse {
      gate
    }
  }
}
"""



In [12]:
plnr = "31924"

data = akv_query_FS_graphql(spørjing, {"plnr": plnr})
print(data)

{'data': {'fagpersonerGittPersonlopenumre': [{'aktiv': True, 'id': 'MzE6MjAzLDMxOTI0', 'arbeidsadresse': {'gate': 'Kronstad'}}]}}


In [ ]:
endring = """
mutation MyMutation($id: ID!, $gate: String) {
  endreArbeidsadresseForFagpersoner(
    input: {id: $id, gate: $gate}
  ) {
    errors {
      ... on UgyldigInput {
        __typename
        message
      }
    }
  }
}
"""

bruker_id = data['data']['fagpersonerGittPersonlopenumre'][0]['id']
ny_gateadresse = "Kronstad"

In [14]:

data = akv_query_FS_graphql(endring, {"id": bruker_id, "gate": ny_gateadresse})
print(data)

{'data': {'endreArbeidsadresseForFagpersoner': {'errors': [{'__typename': 'UgyldigInput', 'message': 'banankake inneholder ugyldige tegn'}]}}}


In [ ]:
spørjing_fleire = """
query MyQuery($antal: Int, $start: String) {
  brukere(filter: {eierOrganisasjonskode: "0203"}, first: $antal, after: $start) {
    pageInfo {
      endCursor
      hasNextPage
    }
    nodes {
      id
    }
  }
}
"""